In [24]:
#INTEGRANTES: LUISA MARÍA HERNÁNDEZ QUINTERO, CAROLINA TABORDA VARGAS
import pandas as pd
import numpy as np
from scipy import stats
import statistics
# Leer archivo Excel y usar la segunda fila como encabezado
df = pd.read_excel("FLIR_groups1and2.xlsx", header=1)

# Verificar que es un DataFrame
print(type(df))         

print(df.head())


<class 'pandas.core.frame.DataFrame'>
  Unnamed: 0  Unnamed: 1 Unnamed: 2    T_CRmax    T_CLmax        T̅_CR  \
0  SubjectID         NaN  T_offset1  Max1R13_1  Max1L13_1  aveAllR13_1   
1   161117-1         NaN       0.58      34.98      35.36        34.44   
2   161117-2         NaN       0.83      34.71      34.51        34.46   
3   161117-3         NaN       0.85      35.70      35.44        35.00   
4   161117-4         NaN       0.90      35.17      35.50        34.25   

         T̅_CL Unnamed: 7 Unnamed: 8 Unnamed: 9  ... Unnamed: 115  \
0  aveAllL13_1      T_RC1  T_RC_Dry1  T_RC_Wet1  ...     aveOralM   
1        34.85      34.91      34.91      34.60  ...        36.59   
2        34.24      34.68      34.68      34.44  ...        37.19   
3        34.78      35.67      35.67      35.46  ...        37.34   
4        35.00      35.14      35.14      35.08  ...        37.09   

  Unnamed: 116 Unnamed: 117               Unnamed: 118 Unnamed: 119  \
0       Gender          Age    

In [7]:
#Análisis exploratorio de datos

#Promedio de la variable Max1R13_
df = pd.read_excel("FLIR_groups1and2.xlsx", header=2)
columna=df[['Max1R13_1', 'Max1R13_2', 'Max1R13_3', 'Max1R13_4']].mean(axis=1)
df_promedio = pd.DataFrame(columna, columns=['promedio_max'])
print(f"El promedio de Max1R13_ es: \n {df_promedio}")

#Tipos de datos: Variables numéricas y Variables categóricas
variables_numericas = ['aveOralM', 'Age', 'T_atm', 'Humidity', 'Max1R13_']
variables_categoricas = ['Gender', 'Ethnicity', 'Cosmetics']



El promedio de Max1R13_ es: 
       promedio_max
0          35.0300
1          34.5500
2          35.6525
3          35.2225
4          35.5450
...            ...
1015       35.6425
1016       35.9825
1017       36.4075
1018       35.8150
1019       35.7075

[1020 rows x 1 columns]


In [ ]:
#Construcción de un DataFrame
#Tipos de datos: Variables numéricas y Variables categóricas
variables_categoricas = ['Gender', 'Ethnicity', 'Cosmetics']

#Tipos de datos: Variables numéricas y Variables categóricas

list_1=df['aveOralM'].tolist()
list_2=df['Gender'].tolist()
list_3=df['Age'].tolist()
list_4=df['Ethnicity'].tolist()
list_5=df['T_atm'].tolist()
list_6=df['Humidity'].tolist()
list_7=df['Cosmetics'].tolist()
list_8=df_promedio['promedio_max'].tolist()

def valores_estadisticos(valores):
    media = np.mean(valores)
    mediana = np.median(valores)
    try:
        moda = statistics.mode(valores)
    except statistics.StatisticsError:
        moda = 'No única'  # Si no hay moda clara
    sd = np.std(valores, ddof=1)  # Desviación estándar muestral
    mad = np.mean(np.abs(valores - np.mean(valores)))  # Desviación absoluta media
    varianza = np.var(valores, ddof=1)
    iqr = np.percentile(valores, 75) - np.percentile(valores, 25)
    cv = sd / media if media != 0 else None  # Evita división por cero
    cvm = mad / media if media != 0 else None
    return media, mediana, moda, sd, mad, varianza, iqr, cv, cvm



        




[35.03, 34.55, 35.6525, 35.2225, 35.545, 35.1325, 35.2825, 35.105000000000004, 35.307500000000005, 35.89, 35.275000000000006, 35.385000000000005, 35.572500000000005, 35.3775, 36.2175, 35.582499999999996, 35.385000000000005, 35.440000000000005, 34.7175, 35.4925, 35.00666666666667, 35.7, 34.535, 34.977500000000006, 35.99, 34.87, 35.629999999999995, 35.0925, 36.184999999999995, 35.055, 35.9075, 35.05, 35.9775, 34.8875, 34.892500000000005, 35.9075, 35.5925, 35.635, 35.1225, 35.114999999999995, 36.1175, 35.5875, 35.7325, 34.410000000000004, 35.4075, 35.6625, 36.027499999999996, 35.845, 36.5325, 34.585, 35.620000000000005, 35.2775, 35.64, 35.0275, 35.7875, 35.334999999999994, 35.6775, 36.0625, 35.335, 36.2475, 35.457499999999996, 35.2825, 35.662499999999994, 36.1575, 35.42, 35.682500000000005, 36.3725, 34.955, 34.95, 36.1225, 34.902499999999996, 35.434999999999995, 37.325, 35.5375, 35.105000000000004, 35.47, 35.696666666666665, 34.995000000000005, 36.836666666666666, 35.71, 35.6225, 37.1575,